In [5]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding, EvalPrediction
from datasets import load_dataset, DatasetDict
import torch
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    matthews_corrcoef
)

In [ ]:
pip install -U torch transformers datasets accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 14.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
model_ckpt = "Alibaba-NLP/gte-base-en-v1.5"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
print(device)

cpu


In [ ]:
dataset = DatasetDict({
    "train": load_dataset("csv", data_files="/content/drive/MyDrive/Automatic-exam-scoring/data/train_under.csv", split="train"),
    "validation": load_dataset("csv", data_files="/content/drive/MyDrive/Automatic-exam-scoring/data/val_under.csv", split="train"),
    "test": load_dataset("csv", data_files="/content/drive/MyDrive/Automatic-exam-scoring/data/test.csv", split="train"),
})

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['corrected_text', 'length', 'ratio_err', 'labels'],
        num_rows: 8923
    })
    validation: Dataset({
        features: ['corrected_text', 'length', 'ratio_err', 'labels'],
        num_rows: 2231
    })
    test: Dataset({
        features: ['corrected_text', 'length', 'ratio_err', 'labels'],
        num_rows: 3462
    })
})


In [ ]:
dataset = dataset.map(
    lambda x: {
        "text" : x["corrected_text"]
    },
    remove_columns = ["corrected_text", "length", "ratio_err"]
)

Map:   0%|          | 0/8923 [00:00<?, ? examples/s]

Map:   0%|          | 0/2231 [00:00<?, ? examples/s]

Map:   0%|          | 0/3462 [00:00<?, ? examples/s]

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['labels', 'text'],
        num_rows: 8923
    })
    validation: Dataset({
        features: ['labels', 'text'],
        num_rows: 2231
    })
    test: Dataset({
        features: ['labels', 'text'],
        num_rows: 3462
    })
})


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True)

In [ ]:
dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/8923 [00:00<?, ? examples/s]

Map:   0%|          | 0/2231 [00:00<?, ? examples/s]

Map:   0%|          | 0/3462 [00:00<?, ? examples/s]

In [ ]:
tokenizer.tokenize("Hello world, how are u ?", add_special_tokens=True)

['[CLS]', 'hello', 'world', ',', 'how', 'are', 'u', '?', '[SEP]']

In [ ]:
tokenizer.pad_token

'[PAD]'

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
dataset = dataset.map(lambda x: {"labels": x["labels"] - 1})

Map:   0%|          | 0/8923 [00:00<?, ? examples/s]

Map:   0%|          | 0/2231 [00:00<?, ? examples/s]

Map:   0%|          | 0/3462 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt,
                                                           trust_remote_code=True,
                                                           num_labels=6).to(device)

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

configuration.py:   0%|          | 0.00/7.13k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- configuration.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling.py:   0%|          | 0.00/57.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- modeling.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/547M [00:00<?, ?B/s]

Some weights of NewForSequenceClassification were not initialized from the model checkpoint at Alibaba-NLP/gte-base-en-v1.5 and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(model)

NewForSequenceClassification(
  (new): NewModel(
    (embeddings): NewEmbeddings(
      (word_embeddings): Embedding(30528, 768, padding_idx=0)
      (rotary_emb): NTKScalingRotaryEmbedding()
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): NewEncoder(
      (layer): ModuleList(
        (0-11): 12 x NewLayer(
          (attention): NewAttention(
            (qkv_proj): Linear(in_features=768, out_features=2304, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (o_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (mlp): NewGatedMLP(
            (up_gate_proj): Linear(in_features=768, out_features=6144, bias=False)
            (down_proj): Linear(in_features=3072, out_features=768, bias=True)
            (act_fn): GELUActivation()
            (hidden_dropout): Dropout(p=0.1, inplace=False)
          )
          (attn_ln): LayerNorm((768,), 

In [ ]:
def compute_metrics(preds: EvalPrediction):
    """
    Compute metrics for the task

    Args:
        preds {EvalPrediction}: the predictions from the model
    Returns:
        dict: a dictionary of metrics
    """

    labels = preds.label_ids
    preds = preds.predictions.argmax(-1)

    acc = accuracy_score(labels, preds)
    weighted_precision = precision_score(labels, preds, average="weighted")
    weighted_recall = recall_score(labels, preds, average="weighted")
    weighted_f1 = f1_score(labels, preds, average="weighted")
    mcc = matthews_corrcoef(labels, preds)

    macro_precision = precision_score(labels, preds, average="macro")
    macro_recall = recall_score(labels, preds, average="macro")
    macro_f1 = f1_score(labels, preds, average="macro")

    micro_precision = precision_score(labels, preds, average="micro")
    micro_recall = recall_score(labels, preds, average="micro")
    micro_f1 = f1_score(labels, preds, average="micro")

    return {
        "accuracy": acc,
        "weighted_precision": weighted_precision,
        "weighted_recall": weighted_recall,
        "weighted_f1": weighted_f1,
        "mcc": mcc,
        "macro_precision": macro_precision,
        "macro_recall": macro_recall,
        "macro_f1": macro_f1,
        "micro_precision": micro_precision,
        "micro_recall": micro_recall,
        "micro_f1": micro_f1
    }

In [ ]:
BATCH_SIZE = 2
LR = 2e-5

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/models_lazy_teacherv2",
    num_train_epochs=6,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_steps=100,
    learning_rate=LR,
    evaluation_strategy="steps",
    bf16=True,
    eval_steps=250,
    report_to="tensorboard",
    lr_scheduler_type="cosine"
)

In [12]:
weights = dataset["train"].to_pandas()["labels"].value_counts(normalize=False).sort_index().values
weights = 1 / weights

In [15]:
weights = torch.tensor(weights)

/var/folders/z0/k_s93jy57s70_3sd68svzdxw0000gn/T/ipykernel_1938/3875099978.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  weights = torch.tensor(weights)


In [16]:
weights

tensor([0.0012, 0.0004, 0.0004, 0.0004, 0.0016, 0.0100], dtype=torch.float64)

In [17]:
class WeightedTrainer(Trainer):

    classes_weights = torch.tensor([0.0012, 0.0004, 0.0004, 0.0004, 0.0016, 0.0100])

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = torch.nn.CrossEntropyLoss(weight=self.classes_weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

In [ ]:
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,Weighted Precision,Weighted Recall,Weighted F1,Mcc,Macro Precision,Macro Recall,Macro F1,Micro Precision,Micro Recall,Micro F1
200,1.389200,1.226781,0.474675,0.369441,0.474675,0.353859,0.324819,0.222643,0.286062,0.213252,0.474675,0.474675,0.474675
400,1.057100,1.032365,0.560287,0.509906,0.560287,0.496324,0.408807,0.363614,0.338219,0.300223,0.560287,0.560287,0.560287
600,0.964000,0.994679,0.573286,0.571211,0.573286,0.567582,0.438403,0.460183,0.473154,0.458968,0.573286,0.573286,0.573286
800,1.001200,0.979695,0.588974,0.598518,0.588974,0.560355,0.444777,0.508246,0.397854,0.394562,0.588974,0.588974,0.588974
1000,0.975000,0.925044,0.595697,0.590682,0.595697,0.573603,0.474004,0.481998,0.472065,0.461442,0.595697,0.595697,0.595697
1200,0.787200,0.938651,0.616764,0.612497,0.616764,0.603432,0.486822,0.516955,0.463617,0.472230,0.616764,0.616764,0.616764
1400,0.784800,0.907431,0.618557,0.617334,0.618557,0.606430,0.486875,0.642230,0.472638,0.504204,0.618557,0.618557,0.618557
1600,0.796700,0.897846,0.616764,0.605398,0.616764,0.599416,0.490463,0.504410,0.463351,0.468540,0.616764,0.616764,0.616764
1800,0.763000,0.848589,0.636038,0.635103,0.636038,0.632128,0.519424,0.600599,0.561416,0.571492,0.636038,0.636038,0.636038
2000,0.798100,0.934657,0.619901,0.614271,0.619901,0.594747,0.491909,0.519566,0.434825,0.444888,0.619901,0.619901,0.619901


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on th

TrainOutput(global_step=6690, training_loss=0.528443263214978, metrics={'train_runtime': 3904.1325, 'train_samples_per_second': 13.713, 'train_steps_per_second': 1.714, 'total_flos': 1.1544821508746772e+16, 'train_loss': 0.528443263214978, 'epoch': 5.997982741230528})

In [ ]:
trainer.save_model("/content/drive/MyDrive/models_lazy_teacherv2")

In [ ]:
trainer.evaluate(eval_dataset=dataset["test"])

{'eval_loss': 2.2573506832122803,
 'eval_accuracy': 0.6184286539572501,
 'eval_weighted_precision': 0.6198043165627963,
 'eval_weighted_recall': 0.6184286539572501,
 'eval_weighted_f1': 0.6161302399276612,
 'eval_mcc': 0.4789192658171589,
 'eval_macro_precision': 0.6167370078213045,
 'eval_macro_recall': 0.5263400421353718,
 'eval_macro_f1': 0.5521962218325465,
 'eval_micro_precision': 0.6184286539572501,
 'eval_micro_recall': 0.6184286539572501,
 'eval_micro_f1': 0.6184286539572501,
 'eval_runtime': 17.2112,
 'eval_samples_per_second': 201.148,
 'eval_steps_per_second': 50.316,
 'epoch': 5.997982741230528}